### Create a simple solar system model.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Define a planet class.

In [ ]:
class planet():
    "A planet in our solar system"
    def __init__(self,semimajor,eccentricity):
        self.x = np.zeros(2) #x and y position
        self.v = np.zeros(2) #x and y velocity
        self.a_g = np.zeros(2) #x and y acceleration
        self.t = 0.0 #current time
        self.dt = 0.0 #current timestep
        self.a = semimajor #semimajor axis of the orbit
        self.e = eccentricity #eccentricity of the orbit
        self.istep = 0 #current integer timestep
        self.name = "Planet Me" #name for the planet
        

### Define a dictionary with some constants.

In [ ]:
solar_system = { "M_sun":1.0, "G":39.4784176043574320}

### Define some functions for setting circular velocity, and acceleration.

In [ ]:
def SolarCircularVelocity(p,solar_system):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = ( p.x[0]**2 + p.x[1]**2 )**0.5
    
    #return the circular velocity
    return (G*M/r)**0.5


### Write a function to compute the gravitational acceleration on each planet from the Sun.

In [ ]:
def SolarGravitationalAcceleration(p,solar_system):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = ( p.x[0]**2 + p.x[1]**2 )**0.5
    
    #acceleration in AU/yr/yr
    a_grav = -1.0*G*M/r**2
    
    #find the angle at this position
    if(p.x[0]==0.0):
        if(p.x[1]>0.0):
            theta = 0.5*np.pi
        else:
            theta = 1.5*np.pi
    else:
        theta = np.arctan2(p.x[1],p.x[0])
        
    #set the x and y components of the velocity
    #p.a_g[0] = a_grav * np.cos(theta)
    #p.a_g[1] = a_grav * np.sin(theta)
    return a_grav*np.cos(theta), a_grav*np.sin(theta)

### Compute the timestep.

In [ ]:
def calc_dt(p):
    
    #integration tolerance
    ETA_TIME_STEP = 0.0004
    
    #compute timestep
    eta = ETA_TIME_STEP
    v = (p.v[0]**2 + p.v[1]**2)**0.5
    a = (p.a_g[0]**2 + p.a_g[1]**2)**0.5
    dt = eta * np.fmin(1./np.fabs(v),1./np.fabs(a)**0.5)
    
    return dt

### Define the initial conditions.

In [ ]:
def SetPlanet(p, i, solar_system):
    
    AU_in_km = 1.495979e+8 #an AU in km
    
    #circular velocity
    v_c = 0.0 #circular velocity in AU/yr
    v_e = 0.0 #velocity at perihelion in AU/yr
    
    #planet-by planet initial conditions
    
    #Mercury
    if(i==0):
        #semi-major axis in AU
        p.a = 57909227.0/AU_in_km
        
        #eccentricity
        p.e = 0.20563593
        
        #name
        p.name = "Mercury"
        
    #Venus
    elif(i==1):
        #semi-major axis in AU
        p.a = 108209475.0/AU_in_km
        
        #eccentricity
        p.e = 0.00677672
        
        #name
        p.name = "Venus"
        
    #Earth
    elif(i==2):
        #semi-major axis in AU
        p.a = 1.0
        
        #eccentricity
        p.e = 0.01671123
        
        #name
        p.name = "Earth"
        
    #set remaining properties
    p.t = 0.0
    p.x[0] = p.a*(1.0-p.e)
    p.x[1] = 0.0
    
    #get equiv circular velocity
    v_c = SolarCircularVelocity(p,solar_system)
    
    #velocity at perihelion
    v_e = v_c*(1 + p.e)**0.5
    
    #set velocity
    p.v[0] = 0.0 #no x velocity at perihelion
    p.v[1] = v_e #y velocity at perihelion (counter clockwise)
    
    #calculat gravitational acceleration from Sun
    p.a_g = SolarGravitationalAcceleration(p,solar_system)
    
    #set timestep
    p.dt = calc_dt(p)
    

### Write leapfrog integrator.

In [ ]:
def x_first_step(x_i, v_i, a_i, dt):
    #x_1/2 = x_0 + 1/2*v_0*Delta_t + 1/4*a_0*Delta_t^2
    return x_i + 0.5*v_i*dt + 0.25*a_i*dt**2

In [ ]:
def v_full_step(v_i, a_ipoh, dt):
    #v_i+1 = v_i + a_i + 1/2*Delta_t
    return v_i + a_ipoh*dt;

In [ ]:
def x_full_step(x_ipoh, v_ipl, a_ipoh, dt):
    #x_3/2 = x_1/2 + v_i+1*Delta_t
    return x_ipoh + v_ipl*dt;

### Write a function to save the date to a file.

In [ ]:
def SaveSolarSystem(p, n_planets, t, dt, istep, ndim):
    
    #loop over the number of planets
    for i in range(n_planets):
        
        #define a filename
        fname = "planet.%s.txt" % p[i].name
        
        if(istep==0):
            #create the file
            fp = open(fname,"w")
        else:
            #append to the file
            fp = open(fname,"a")
            
        #compute the drifted properties of the planet
        v_drift = np.zeros(ndim)
        
        for k in range(ndim):
            v_drift[k] = p[i].v[k] + 0.5*p[i].a_g[k]*p[i].dt
            
        #write the data to file
        s = "%6d\t%6.5f\t%6.5f\t%6d\t%6.5f\t%6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\n" % \
        (istep,t,dt,p[i].istep,p[i].t,p[i].dt,p[i].x[0],p[i].x[1],v_drift[0],v_drift[1],p[i].a_g[0],p[i].a_g[1])
        
        #write the string to file as a new line
        fp.write(s)
        
        #close the file
        fp.close()
        
        

### Write a function to evolve the solar system.

In [ ]:
def EvolveSolarSystem(p,n_planets,t_max,solar_system):
    
    #number of dimensions
    ndim = 2
    
    #define the first timestep
    dt = 0.5/365.25
    
    #define the starting time
    t = 0.0
    
    #define the starting step
    istep = 0
    
    #save the initial conditions
    SaveSolarSystem(p,n_planets,t,dt,istep,ndim)
    
    #begin a loop over the global timescale
    while(t<t_max):
        
        #limit time to tmax
        if(t+dt>t_max):
            dt = t_max - t #limit step to tmax
            
        #evolve each planet
        for i in range(n_planets):
            
            while(p[i].t<t+dt):
                
                #special case for istep==0
                if(p[i].istep==0):
                    
                    #take the first step with a verlet scheme
                    for k in range(ndim):
                        p[i].x[k] = x_first_step(p[i].x[k],p[i].v[k],p[i].a_g[k],p[i].dt)
                        
                    #update the acceleration
                    p[i].a_g = SolarGravitationalAcceleration(p[i],solar_system)
                    
                    #update the time by 1/2 step
                    p[i].t += 0.5*p[i].dt
                    
                    #update the timestep
                    p[i].dt = calc_dt(p[i])
                    
                #contunue with a normal step
                
                #limit to align with the global timestep
                if(p[i].t + p[i].dt > t+dt):
                    p[i].dt = t+dt-p[i].t
                    
                #evolve the velocity
                for k in range(ndim):
                    p[i].v[k] = v_full_step(p[i].v[k],p[i].a_g[k],p[i].dt)
                    
                #evolve the position
                for k in range(ndim):
                    p[i].x[k] = x_full_step(p[i].x[k],p[i].v[k],p[i].a_g[k],p[i].dt)
                
                
                #update the acceleration
                p[i].a_g = SolarGravitationalAcceleration(p[i],solar_system)
                
                #update by dt
                p[i].t += p[i].dt
                
                #compute a new timestep
                p[i].dt = calc_dt(p[i])
                
                #update the planet's timestep
                p[i].istep += 1
                
        #now update the global system time
        t += dt
        #update the global step number
        istep += 1
        
        #output the current state
        SaveSolarSystem(p,n_planets,t,dt,istep,ndim)
        
    #print the final steps and time
    print("Time t = ",t)
    
    print("Maximum t = ",t_max)
    print("Maximum number of steps = ",istep)
    
    #end of evolution

### Perform the integration of the solar system.

In [ ]:
#set the number of planets
n_planets = 3

#set the maximum time of the simulation
t_max = 2.0

#create an empty list of planets
p = []

#create the planets
for i in range(n_planets):
    
    #create an empty planet
    ptmp = planet(0,0)
    
    #set the planet properties
    SetPlanet(ptmp,i,solar_system)
    
    #remember the planet
    p.append(ptmp)
    

#evolve the solar system
EvolveSolarSystem(p,n_planets,t_max,solar_system)

### Create a routine to read in the data.

In [ ]:
def read_twelve_arrays(fname):
    
    fp = open(fname,"r")
    
    f1 = fp.readlines()
    
    n = len(f1)
    
    a = np.zeros(n)
    b = np.zeros(n)
    c = np.zeros(n)
    d = np.zeros(n)
    f = np.zeros(n)
    g = np.zeros(n)
    h = np.zeros(n)
    j = np.zeros(n)
    k = np.zeros(n)
    l = np.zeros(n)
    m = np.zeros(n)
    p = np.zeros(n)
    for i in range(n):
        a[i] = float(f1[i].split()[0])
        b[i] = float(f1[i].split()[1])
        c[i] = float(f1[i].split()[2])
        d[i] = float(f1[i].split()[3])
        f[i] = float(f1[i].split()[4])
        g[i] = float(f1[i].split()[5])
        h[i] = float(f1[i].split()[6])
        j[i] = float(f1[i].split()[7])
        k[i] = float(f1[i].split()[8])
        l[i] = float(f1[i].split()[9])
        m[i] = float(f1[i].split()[10])
        p[i] = float(f1[i].split()[11])
        
    return a,b,c,d,f,g,h,j,k,l,m,p

### Read the data back in.

In [ ]:
fname = "planet.Mercury.txt"
istepMg, tMg, dtMg, istepM, tM, dtM, xM, yM, vxM, vyM, axM, ayM = read_twelve_arrays(fname)

In [ ]:
fname = "planet.Venus.txt"
istepVg, tVg, dtVg, istepV, tV, dtV, xV, yV, vxV, vyV, axV, ayV = read_twelve_arrays(fname)

In [ ]:
fname = "planet.Earth.txt"
istepEg, tEg, dtEg, istepE, tE, dtE, xE, yE, vxE, vyE, axE, ayE = read_twelve_arrays(fname)

### Plot the data.

In [ ]:
fig = plt.figure(figsize=(7,7))

xSun= np.zeros(1)
ySun= np.zeros(1)

plt.plot(xSun,ySun, 'o', color= "orange", label="Sun")

plt.plot(xM,yM,color="red")
plt.plot(xM[-1],yM[-1],'o',color="red",label="Mercury")

plt.plot(xV,yV,color="green")
plt.plot(xV[-1],yV[-1],'o',color="green",label="Venus")

plt.plot(xE,yE,color="blue")
plt.plot(xE[-1],yE[-1],'o',color="blue",label="Earth")

plt.xlim([-1.25,1.25])
plt.ylim([-1.25,1.25])
plt.xlabel('x [AU]')
plt.ylabel('y [AU]')
#plt.axes().set_aspect('equal')
plt.legend(frameon=False,loc=2)

In [ ]:
print(yM)

In [ ]:

n_frames = len(xM)

n_skip = 10

k = 0

for i in range(0,n_frames,n_skip):

    fig = plt.figure(figsize=(7,7))

    xSun= np.zeros(1)
    ySun= np.zeros(1)

    plt.plot(xSun,ySun, 'o', color= "orange", label="Sun")

    plt.plot(xM[:i+1],yM[:i+1],color="red")
    plt.plot(xM[i],yM[i],'o',color="red",label="Mercury")

    plt.plot(xV[:i+1],yV[:i+1],color="green")
    plt.plot(xV[i],yV[i],'o',color="green",label="Venus")

    plt.plot(xE[:i+1],yE[:i+1],color="blue")
    plt.plot(xE[i],yE[i],'o',color="blue",label="Earth")

    s = "t = %5.4f" % tMg[i]
    plt.text(0.9,0.9,s)
    plt.xlim([-1.25,1.25])
    plt.ylim([-1.25,1.25])
    plt.xlabel('x [AU]')
    plt.ylabel('y [AU]')
    plt.gca().set_aspect('equal')
    plt.legend(frameon=False,loc=2)
    fname = "frame.%03d.png" % k
    plt.savefig(fname,bbox_inches='tight',dpi=400)
    plt.clf()
    print(k)
    k += 1 